In [1]:
import utils
import numpy as np
import math
import matplotlib.pyplot as plt

from scipy.signal import medfilt
from plotly import tools
import chart_studio.plotly as py
import plotly.graph_objs as go
import pose
from pose_parser import parse_file

In [8]:
ps = parse_file("dataset/bicep/front_bicep.npy")

Data shape:  (319, 19, 3)
Mean torso:  86.7161602355936


In [3]:
frames = ps
# right_upper_arm_vecs = np.array([pose.Joint.get_vector(frame.rshoulder, frame.relbow) for frame in frames])
# right_lower_arm_vecs = np.array([pose.Joint.get_vector(frame.relbow, frame.rwrist) for frame in frames])

# left_upper_arm_vecs = np.array([pose.Joint.get_vector(frame.lshoulder, frame.lelbow) for frame in frames])
# left_lower_arm_vecs = np.array([pose.Joint.get_vector(frame.lelbow, frame.lwrist) for frame in frames])

# left_upper_arm_vecs = left_upper_arm_vecs/ np.expand_dims(np.linalg.norm(left_upper_arm_vecs, axis=1), axis=1)
# left_lower_arm_vecs = left_lower_arm_vecs/ np.expand_dims(np.linalg.norm(left_lower_arm_vecs, axis=1), axis=1)

# right_upper_arm_vecs = right_upper_arm_vecs/ np.expand_dims(np.linalg.norm(right_upper_arm_vecs, axis=1), axis=1)
# right_lower_arm_vecs = right_lower_arm_vecs/ np.expand_dims(np.linalg.norm(right_lower_arm_vecs, axis=1), axis=1)

# left_upper_arm_forearm_angle = np.degrees(np.arccos(np.clip(np.sum(np.multiply(left_upper_arm_vecs, left_lower_arm_vecs), axis=1), -1.0, 1.0)))
# right_upper_arm_forearm_angle = np.degrees(np.arccos(np.clip(np.sum(np.multiply(right_upper_arm_vecs, right_lower_arm_vecs), axis=1), -1.0, 1.0)))

# for i in range (0,300):
#     print(str(i) + "      " + str(left_upper_arm_forearm_angle[i])+ "     " + str(right_upper_arm_forearm_angle[i]))

In [9]:
frames = ps
left_upperarm_forearm_angles = []
right_upperarm_forearm_angles = []
dire = ["Up"]
left_upperarm = pose.Part(frames[0].lelbow, frames[0].lshoulder)
for i,frame in enumerate(frames):
    right_upperarm = pose.Part(frame.relbow, frame.rshoulder)
    right_forearm = pose.Part(frame.relbow, frame.rwrist)
    left_upperarm = pose.Part(frame.lelbow, frame.lshoulder)
    left_forearm = pose.Part(frame.lelbow, frame.lwrist)
    
    # vec1 = vert_vector
    # vec2 = left_forearm.get_vector()
    # vec1 = vec1 / np.linalg.norm(vec1)
    # vec2 = vec2 / np.linalg.norm(vec2)

    # angle = np.degrees(np.arccos(np.clip(np.dot(vec1, vec2), -1.0, 1.0)))
    left_angle = left_upperarm.calculate_angle(left_forearm)
    right_angle = right_upperarm.calculate_angle(right_forearm)
    left_upperarm_forearm_angles.append(left_angle)
    right_upperarm_forearm_angles.append(right_angle)
    left_upperarm_forearm_angles_filtered= medfilt(left_upperarm_forearm_angles, 19)
    right_upperarm_forearm_angles_filtered = medfilt(right_upperarm_forearm_angles,19)
    if i!=0:
        prev_left_angle = left_upperarm_forearm_angles_filtered[i-1]
        current_left_angle = left_upperarm_forearm_angles_filtered[i]
        if (prev_left_angle > current_left_angle):
            dire.append("Up")
        else:
            dire.append("Down")
    


In [5]:
left_upperarm_forearm_angles= medfilt(left_upperarm_forearm_angles, 19)
right_upperarm_forearm_angles = medfilt(right_upperarm_forearm_angles,19)
print(dire)

['Up', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Up', 'Down', 'Down', 'Down', 'Down', 'Up', 'Up', 'Down', 'Up', 'Up', 'Down', 'Down', 'Up', 'Down', 'Down', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Up', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Up', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Up', 'Down', 'Up', 'Up', 'Down', 'Down', 'Down', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up',

In [6]:
import cv2
import numpy as np
import time

In [12]:
cap = cv2.VideoCapture('videos/bicep.mp4')
ps2 = parse_file("dataset/bicep/front_bicep.npy", normalize=False)
if(cap.isOpened()==False):
    print("Error")
i= 0

while(cap.isOpened()):
    ret, frame = cap.read()
    if (ret== True):
        time.sleep(0.05)
        for name, joint in ps2[i]:
            x = int(joint.x)
            y = int(joint.y)
            cv2.circle(frame, (x, y), 2, (0, 255, 255), 2)
        text = str(round(left_upperarm_forearm_angles[i],2))+ "  " + str(round(right_upperarm_forearm_angles[i],2)) + " " + dire[i]
        cv2.putText(frame, text, (10,50),cv2.FONT_HERSHEY_COMPLEX,0.6,(0,255,255),2)
        cv2.imshow('Frame', frame)
        i=i+1
        if (cv2.waitKey(25) & 0xFF == ord('q')):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()

Data shape:  (319, 19, 3)
